## Explain

```SQL
CREATE TABLE temp (id serial, name text);
INSERT INTO temp (name) SELECT 'hans' FROM generate_series(1, 2000000);
INSERT INTO temp (name) SELECT 'paul' FROM generate_series(1, 2000000);
```

```SQL
SELECT name, count(*) FROM temp GROUP BY 1;
```

### Sequential Scan
- Cost 비용은 43455.43
- 이 비용은 GPU 또는 CPU 자원 소모를 의미 함. 

```SQL
EXPLAIN SELECT * FROM temp WHERE id = 432332;
                              QUERY PLAN                               
-----------------------------------------------------------------------
 Gather  (cost=1000.00..43455.43 rows=1 width=9)
   Workers Planned: 2
   ->  Parallel Seq Scan on temp  (cost=0.00..42455.33 rows=1 width=9)
         Filter: (id = 432332)
(4 rows)
```

### Simple Index
- index를 생성하여 탐색을 진행한다. 

```SQL
postgres=# CREATE INDEX idx_id ON temp (id);
CREATE INDEX
```

- 시간 측정하는 명령어를 입력한다. 

```SQL
postgres=# \timing
Timing is on.
postgres=# SELECT * FROM temp WHERE id = 43242;
  id   | name 
-------+------
 43242 | hans
(1 row)

Time: 1.216 ms
```

- 그러나 index를 사용하는 것은 공짜가 아니다. 

```SQL
postgres=# \di+
                                             List of relations
 Schema |        Name         | Type  |  Owner   |     Table      | Persistence |    Size    | Description 
--------+---------------------+-------+----------+----------------+-------------+------------+-------------
 public | actors_pkey         | index | postgres | actors         | permanent   | 16 kB      | 
 .
 .
 public | idx_id              | index | postgres | temp           | permanent   | 86 MB 
```

### index scan 

```SQL
postgres=# EXPLAIN SELECT * FROM temp ORDER BY id DESC LIMIT 10;
                                         QUERY PLAN                                          
---------------------------------------------------------------------------------------------
 Limit  (cost=0.43..0.74 rows=10 width=9)
   ->  Index Scan Backward using idx_id on temp  (cost=0.43..125505.43 rows=4000000 width=9)
(2 rows)
```

### bitmap Scans
- 이번에는 다음 코드를 확인한다. 
- 비트맵 탐색은 인덱스에서 조회한 내용을 잠시 비트맵에 저장한 후 비트맵에 저장된 내용을 기반으로 테이블에서 원하는 결과를 가져오는 작업. 

```SQL
postgres=# Explain SELECT * FROM temp WHERE id = 30 OR id = 50;
                                QUERY PLAN                                 
---------------------------------------------------------------------------
 Bitmap Heap Scan on temp  (cost=8.88..16.85 rows=2 width=9)
   Recheck Cond: ((id = 30) OR (id = 50))
   ->  BitmapOr  (cost=8.88..8.88 rows=2 width=0)
         ->  Bitmap Index Scan on idx_id  (cost=0.00..4.44 rows=1 width=0)
               Index Cond: (id = 30)
         ->  Bitmap Index Scan on idx_id  (cost=0.00..4.44 rows=1 width=0)
               Index Cond: (id = 50)
(7 rows)

Time: 2.790 ms
```